1차적인 전처리를 진행한 코드입니다.

# raw데이터에 필요한 외부데이터 추가

## 날씨 외부데이터 추가
1. 날씨 관련 데이터 추가: 기온, 강수량,vs,lcsch,dc10tca,dsnw,icsr,ss,pa,pv,hm,ws
2. 102개 관측소에서 1시간 단위로 측정한 날씨 데이터가 주어지는데, 지역 정보는 활용할 수 없으므로 102개 지역의 데이터는 평균 내어 이용했습니다.
3. 또한, 제공받을 수 있었던 피쳐 중 일부 피쳐를 임의로 선택하여 이용했습니다. 날씨가 홈쇼핑의 시청률 또는 판매량에 영향을 끼쳤다면 시각적인 요소가 결정적이었을 것으로 판단했고 제공받은 날씨 데이터 중 가정 내에서 창문 등을 통해 시각적으로 확인하기 쉬울 것이라고 생각되는 요소만을 1차적으로 선택했습니다.
4. raw_trian, raw_test 파일에 미리 추가했습니다.

변수명|설명
--|--
기온|기온
강수량|강수량
vs| 시정(가시거리) 
lcsch| 최저 운고(구름의 양 관련), 구름 밑부분까지의 고도. 최저 운고가 낮을수록 구름이 낮게 위치한다. 
dc10tca| 전운량(구름의 양 관련), 하늘에 구름이 덮고 있는 비율에 따라 0-10사이의 수를 부여, 전부 구름이면 10, 구름이 없으면 0
dsnw| 적설 지면에 쌓인 눈의 깊이. 기간과 관계없이 지면에 쌓여있는 눈의 깊이만을 의미, 관측소 주위의 지면이 1/2 이상 눈으로 덮여있어야 적설이 있다고 봄.
icsr| 일사량 태양으로부터 오는 태양 복사 에너지가 지표에 닿는 양. 태양의 고도가 높을수록 일사량이 증가하며, 태양이 천정에 위치할 때 일사량은 최대가 된다. 
ss| 일조량 태양 광선이 지면에 얼마동안 비추었는지를 의미.
pa| 기압 
pv| 증기압 
hm| 습도 
ws| 풍속
|(출처: [기상청](https://www.data.go.kr/data/15043648/fileData.do))


## 타 방송 관련 데이터 추가 
1. 타 방송 관련 데이터 추가: morning_drama, popular_program  
2. <img src="사진\닐슨.png" alt="닐슨" style="zoom:20%;" />
  닐슨에서 위의 이미지와 같이 일별 시청률 순위를 확인할 수 있습니다. 2019년 1월 1일부터 2019년 12월 31일의 일별 시청률 데이터를 통해 프로그램별 평균 시청률을 계산해본 후, 평균 시청률이 높은 순으로 sorting하여 각 프로그램들의 시간대를 살펴보았습니다.  같은 시간대인 경우 평균 시청률 순위가 비슷한 것을 확인했습니다.(시청률 평균 순위가 높았던 프로그램이 종영한 후 동시간대에 후속으로 방영되는 프로그램의 경우 역시 시청률 평균 순위가 높았음). 이러한 시청률이 홈쇼핑 구매에 영향을 줄것이라 판단해 추가했습니다.
3. raw_trian, raw_test 파일에 미리 추가했습니다.

변수명|설명
--|--
popular_program|평균 시청률이 7% 이상인 프로그램들이 끝난 직후에 방송하는 경우 1, 아닌 경우 0을 부여함. 
morning_drama|홈쇼핑 시장의 주 고객층이 40-50대 여성임을 고려하여, KBS 아침마당, SBS 아침드라마 시간대 직후인 경우 1, 아니면 0을 부여함. (MBC, KBS 아침드라마는 작년에 폐지됨)
|[닐슨](https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=01)



In [1]:
import warnings
warnings.filterwarnings(action='ignore') 
import numpy as np
import pandas as pd
train = pd.read_excel("data/raw_train.xlsx", skiprows =1)
test = pd.read_excel("data/raw_test.xlsx", skiprows =1)

## holiday 외부 데이터 추가
1. 공휴일, 기념일과 같은 정보를 나타내는 컬럼을 추가했습니다.

In [2]:
holiday = pd.read_csv("data/external data/holiday.csv")
holiday = holiday[holiday["유형"].apply(lambda x: False if x == "Season" else True)].reset_index(drop = True)
tmp = dict(zip(holiday["날짜"], holiday["유형"]))

train["holiday"] = train["방송일시"].apply(lambda x: tmp[str(x).split(" ")[0]] if str(x).split(" ")[0] in tmp else np.nan )
test["holiday"] = test["방송일시"].apply(lambda x: tmp[str(x).split(" ")[0]] if str(x).split(" ")[0] in tmp else np.nan )

# raw데이터에 필요한 수작업 columns 추가

## 브랜드 수작업 컬럼 추가
1. 각 제품명을 확인해서 브랜드를 추가했습니다.
2. test data의 브랜드는 train 데이터에 있는 경우만 추가했습니다.
3. raw_trian, raw_test 파일에 미리 추가했습니다.

## 지역, 광고 사람 수작업 컬럼 추가
1. 각 제품명에서 지역과, 광고하는 사람을 확인해서 직접 추가했습니다.
2. raw_trian, raw_test 파일에 미리 추가했습니다.

# 필요 없는 row 제거
판매단가 0인 상품, 취급액 np.nan인 상품을 모두 삭제했습니다.

In [3]:
# 판매단가 0인 상품, 취급액 np.nan인 상품
train = train[ (train["판매단가"] != 0) & (train["취급액"].notna())].reset_index(drop=True)
test = test[test["판매단가"] != 0].reset_index(drop=True)

# 상품군, 마더코드, 상품코드 & 상품명 관련 전처리

## 띄어쓰기 된 상품명
상품명에 띄어쓰기가 제대로 돼있지 않은 경우가 있어서 konlpy 패키지를 이용해 띄어쓰기를 했습니다.

In [4]:
from konlpy.tag import Okt
import re
from tqdm import tqdm
tqdm.pandas()
# 띄어쓰기
okt = Okt()
def spacing_okt(wrongSentence):
    wrongSentence = "".join([i for i in list(wrongSentence) if i.isalnum() or i in [" "]])
    re.sub(r'[^\d\s\w]', ' ', wrongSentence)
    wrongSentence = " ".join(wrongSentence.split())
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected
train["상품명_띄어쓰기"] = train["상품명"].progress_apply(spacing_okt)
test["상품명_띄어쓰기"] = test["상품명"].progress_apply(spacing_okt)

train["상품명_띄어쓰기_set"] = train["상품명_띄어쓰기"].apply(lambda x: " ".join(set(x.split())))
test["상품명_띄어쓰기_set"] = test["상품명_띄어쓰기"].apply(lambda x: " ".join(set(x.split())))

100%|████████████████████████████████████████████████████████████████████████████| 2716/2716 [00:01<00:00, 1525.64it/s]


## 상품군, 마더코드, 상품코드 통일
1. 상품명이 동일한데, 상품군, 마더코드, 상품코드가 다른경우가 있습니다. 띄어쓰기 된 상품명 바탕으로 단어 set이 동일할경우 같은 상품군, 상품코드, 마더코드를 반영해 줍니다.
2. 여러개의 상품군, 마더코드, 상품코드 중에 가장 자주등장한 대표 1개를 선정합니다. (동률일 경우 먼저 나온것을 선택했습니다.)

In [5]:
name_codes = []
for name in ["상품군", "마더코드", "상품코드"]:
    data = pd.concat([train,test])
    tmp = data.groupby(["상품명_띄어쓰기_set"])[name].apply(lambda x: x.value_counts().argmax())
    name_codes.append(tmp)
    train[name] = train["상품명_띄어쓰기_set"].map(dict(tmp))
    test[name] = test["상품명_띄어쓰기_set"].map(dict(tmp))

# 밑에서 유사도 정보를 계산하기위해 존재합니다.
name_code = pd.concat(name_codes, axis = 1).reset_index()

##  상품명간의 유사도 계산
1. train data에 없는 test data의 상품군, 상품코드, 마더코드가 많습니다. 
2. 이 문제를 해결하기 위해 이름의 유사도를 정보를 사용했습니다. 학습을 시킬때 원래의 상품코드로 학습시키고 또, 유사도가 가장 높은 상품코드로도 학습을 시켜(유사도를 바탕으로 normalize를 합니다), test에 원래 상품코드가 없는 경우 유사도 정보를 바탕으로 예측을 진행 할 수 있도록 했습니다.
3. 상품명_띄어쓰기 변수를 바탕으로 단어들의 유사도(전체 단어 대비, 단어 중복 개수)를 계산했고 이 정보를 저장했습니다.

In [6]:
name_sim = pd.DataFrame(0,columns = name_code["상품명_띄어쓰기_set"], index = name_code["상품명_띄어쓰기_set"])
name_list = name_code["상품명_띄어쓰기_set"]

def cal_sim(x,ref):
    lst1, lst2 = np.array(x.split()), np.array(ref.split())
    sim = (np.isin(lst1, lst2)).sum()
    return sim / (len(lst1) + len(lst2) - sim)
for col_name in tqdm(name_list):
    name_sim[col_name] = name_list.apply(cal_sim, ref = col_name).values

100%|██████████████████████████████████████████████████████████████████████████████| 1966/1966 [02:09<00:00, 15.20it/s]


# 저장

In [7]:
train.to_csv("data/preprocess1_train.csv")
test.to_csv("data/preprocess1_test.csv")
name_code.to_csv("data/name_code.csv")
name_sim.to_csv("data/name_sim.csv")